In [300]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [301]:
album_data = pd.read_csv('album_data.csv')
album_data.head(5)

,Album_ID,Album_Name,Album_Artists,Album_Songs,Album_Song_IDs,Song_Lyrics,Genius_URLs
0,7mzrIsaAjnXihW3InKjlC3,Taylor Swift,Taylor Swift,Tim McGraw,0Om9WAB5RS09L80DyOfTNa,[Verse 1]\nHe said the way my blue eyes shined...,https://genius.com/Taylor-swift-tim-mcgraw-lyrics
1,7mzrIsaAjnXihW3InKjlC3,Taylor Swift,Taylor Swift,Picture To Burn,32mVHdy0bi1XKgr0ajsBlG,"[Verse 1]\nState the obvious, I didn't get my ...",https://genius.com/Taylor-swift-picture-to-bur...
2,7mzrIsaAjnXihW3InKjlC3,Taylor Swift,Taylor Swift,Teardrops On My Guitar - Radio Single Remix,7zMcNqs55Mxer82bvZFkpg,[Verse 1]\nDrew looks at me\nI fake a smile so...,https://genius.com/Taylor-swift-teardrops-on-m...
3,7mzrIsaAjnXihW3InKjlC3,Taylor Swift,Taylor Swift,A Place in this World,73OX8GdpOeGzKC6OvGSbsv,"[Verse 1]\nI don't know what I want, so don't ...",https://genius.com/Taylor-swift-a-place-in-thi...
4,7mzrIsaAjnXihW3InKjlC3,Taylor Swift,Taylor Swift,Cold As You,7an1exwMnfYRcdVQm0yDev,[Verse 1]\nYou have a way of coming easily to ...,https://genius.com/Taylor-swift-cold-as-you-ly...


In [302]:
OHCO = ['album_id','song_id','verse_num','token_num']

In [303]:
album_data = album_data[['Album_Artists','Album_Name','Album_Songs', 'Song_Lyrics']]
album_data = album_data.set_index(['Album_Name','Album_Songs'])

In [304]:
LIB = album_data.copy()
LIB['doc_id'] = LIB.index
LIB['char_len'] = LIB['Song_Lyrics'].str.len()

# Summary stats
num_docs = LIB.shape[0]
avg_doc_len = LIB['char_len'].mean()

In [310]:
LIB

Album_Artists  \
Album_Name   Album_Songs                                                 
Taylor Swift Tim McGraw                                   Taylor Swift   
             Picture To Burn                              Taylor Swift   
             Teardrops On My Guitar - Radio Single Remix  Taylor Swift   
             A Place in this World                        Taylor Swift   
             Cold As You                                  Taylor Swift   
...                                                                ...   
143          ALL THE LOVE                                   Katy Perry   
             NIRVANA                                        Katy Perry   
             ARTIFICIAL (feat. JID)                         Katy Perry   
             TRUTH                                          Katy Perry   
             WONDER                                         Katy Perry   

                                                                                                Song_Lyrics  \
Album_Name   Album_Songs                                                                                      
Taylor Swift Tim McGraw                                   [Verse 1]\nHe said the way my blue eyes shined...   
             Picture To Burn                              [Verse 1]\nState the obvious, I didn't get my ...   
             Teardrops On My Guitar - Radio Single Remix  [Verse 1]\nDrew looks at me\nI fake a smile so...   
             A Place in this World                        [Verse 1]\nI don't know what I want, so don't ...   
             Cold As You                                  [Verse 1]\nYou have a way of coming easily to ...   
...                                                                                                     ...   
143          ALL THE LOVE                                 [Verse 1]\nI had accepted love wasn't for me\n...   
             NIRVANA                                      [Intro]\nDancing in the blurry lights\nFound y...   
             ARTIFICIAL (feat. JID)                       [Intro: Katy Perry]\nWho's gonna save us\nFrom...   
             TRUTH                                        [Intro]\nI wanna know the truth\nEven if it hu...   
             WONDER                                       [Intro: Tius Sundberg]\nOne day, when we're ol...   

                                                                                                     doc_id  \
Album_Name   Album_Songs                                                                                      
Taylor Swift Tim McGraw                                                          (Taylor Swift, Tim McGraw)   
             Picture To Burn                                                (Taylor Swift, Picture To Burn)   
             Teardrops On My Guitar - Radio Single Remix  (Taylor Swift, Teardrops On My Guitar - Radio ...   
             A Place in this World                                    (Taylor Swift, A Place in this World)   
             Cold As You                                                        (Taylor Swift, Cold As You)   
...                                                                                                     ...   
143          ALL THE LOVE                                                               (143, ALL THE LOVE)   
             NIRVANA                                                                         (143, NIRVANA)   
             ARTIFICIAL (feat. JID)                                           (143, ARTIFICIAL (feat. JID))   
             TRUTH                                                                             (143, TRUTH)   
             WONDER                                                                           (143, WONDER)   

                                                          char_len  
Album_Name   Album_Songs                                            
Taylor Swift Tim McGraw                                       1870  

In [305]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def tokenize(text):
    # Use regex to find words only (no punctuation, no numbers)
    tokens = re.findall(r"\b[a-zA-Z]+\b", text.lower())
    return tokens

def split_lyrics_to_verses(lyrics):
    """Split a song's lyrics into sections by headers like [Verse], [Chorus], etc."""
    pattern = r'\[.*?\]'
    matches = list(re.finditer(pattern, lyrics))
    
    verses = []
    if not matches:
        return [{'section': 'Full', 'text': lyrics}]
    
    for i, match in enumerate(matches):
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(lyrics)
        section_name = match.group().strip("[]")
        section_text = lyrics[start:end].strip()
        if section_text:
            verses.append({'section': section_name, 'text': section_text})
    return verses

def build_ohco_corpus(df):
    corpus = []

    for (album_id, song_id), row in df.iterrows():
        lyrics = row['Song_Lyrics']
        verses = split_lyrics_to_verses(lyrics)
        for verse_num, verse in enumerate(verses):
            raw_tokens = nltk.word_tokenize(verse['text'])
            raw_tokens = [tokenize(x)[0] for x in raw_tokens if tokenize(x)]
            tagged = nltk.pos_tag(raw_tokens)
        
            for token_num, (token_str, pos) in enumerate(tagged):
                corpus.append({
                    'album_id': album_id,
                    'song_id': song_id,
                    'verse_num': verse_num,
                    'token_num': token_num,
                    'token_str': token_str,
                    'pos': pos,
                    'pos_group': pos[:2]

                })

    return pd.DataFrame(corpus)

In [306]:
CORPUS = build_ohco_corpus(album_data)
CORPUS.sample(100)

,album_id,song_id,verse_num,token_num,token_str,pos,pos_group
763756,"good kid, m.A.A.d city",Real,9,14,and,CC,CC
105575,SOS Deluxe: LANA,Open Arms (feat. Travis Scott),2,12,bad,JJ,JJ
548839,The Eminem Show,Sing For The Moment,0,142,his,PRP$,PR
36615,1989 (Taylor's Version) [Deluxe],Blank Space (Taylor's Version),1,102,space,NN,NN
132957,Beauty Behind The Madness,Angel,1,40,to,TO,TO
...,...,...,...,...,...,...,...
3186,Taylor Swift,Our Song,1,55,and,CC,CC
760821,"good kid, m.A.A.d city","Sing About Me, I'm Dying Of Thirst",3,131,seen,VBN,VB
447870,Scorpion,In My Feelings,6,0,you,PRP,PR
777677,To Pimp A Butterfly,Mortal Man,12,476,that,WDT,WD


In [307]:
CORPUS['term_str'] = CORPUS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = CORPUS['term_str'].value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str', 'pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str', 'pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

df_by_term = CORPUS[['term_str', 'song_id']].drop_duplicates().groupby('term_str').size().to_frame('df')

N_docs = CORPUS['song_id'].nunique()

df_by_term['idf'] = np.log2(N_docs / df_by_term['df'])

df_by_term['dfidf'] = df_by_term['df'] * df_by_term['idf']

VOCAB = VOCAB.join(df_by_term[['df', 'idf', 'dfidf']], on='term_str')
VOCAB

,n,p,i,max_pos,max_pos_group,df,idf,dfidf
term_str,,,,,,,,
a,16612,0.017221,5.859729,DT,DT,1787,0.134050,239.547173
aa,1,0.000001,19.879667,NN,NN,1,10.937374,10.937374
aaaaahhh,1,0.000001,19.879667,NN,NN,1,10.937374,10.937374
aaahhh,1,0.000001,19.879667,NN,NN,1,10.937374,10.937374
aah,11,0.000011,16.420235,JJ,JJ,4,8.937374,35.749495
...,...,...,...,...,...,...,...,...
zs,1,0.000001,19.879667,NN,NN,1,10.937374,10.937374
zucchinis,3,0.000003,18.294705,NN,NN,3,9.352411,28.057234
zulu,9,0.000009,16.709742,NN,NN,4,8.937374,35.749495


In [308]:
VOCAB.sort_values(by='dfidf', ascending=False)['dfidf'].head(20)

term_str
want     1040.776569
come     1040.776569
tell     1040.774921
too      1040.721799
feel     1040.687957
then     1040.471310
been     1040.470074
are      1040.269397
need     1039.958959
gon      1039.838857
there    1039.793427
way      1039.599335
right    1039.502011
wan      1039.313841
take     1038.828701
could    1037.763556
ve       1037.763556
who      1036.632971
off      1036.224884
think    1035.796974
Name: dfidf, dtype: float64

In [309]:
len(album_data)

1992

In [311]:
CORPUS.to_csv('corpus.csv', index=False,sep= '|')
LIB.to_csv('lib.csv',index=False,sep='|')
VOCAB.to_csv('vocab.csv',index=False,sep='|')